In [1]:
from functions import *
from torch.optim import Adam
from functions_pt import *

In [2]:
embed_size = 128
dropout = 0.1
multiple_factor = 1
layers = 2
batch_size = 64
agg = 'mean' # mean,sum,min,max
model = pointNet(layers,embed_size,agg,dropout,multiple_factor)#.to('cuda')
#model.load_state_dict(torch.load('baseline_noBN.pt'))
#model.eval()
model = model.to('cuda')

In [3]:
words_embed = np.load('possible_words_embed.npy')

RL Loop

In [4]:
# model trained from start
lr = 6e-5
clip = 0.03
epochs = 3
damp = 1
iterations = 50
max_data = 2000
states = set()
opt = Adam(model.parameters(),betas=(0.85, 0.95),lr=lr)
for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_save(matrix,np.arange(2309),policy_model,2309,0,model=model,words_embed=words_embed,top=0.6,
                         eps=0.08/(i+1))
    print('iter:{}, score:{}'.format(i,score))
    
    # prepare data for training
    index,val,log_prob = zip(*out)
    for element in index:
        states.add(tuple(element))
    print('set size:{}'.format(len(states)))
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],dtype=torch.float32,device='cuda')
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')

    
    # policy update
    model.train()
    for _ in range(epochs):
        yhat = model((words,length))
        loss = torch.sum(prob*(ys-yhat)**2)/torch.sum(prob)
        loss.backward()
        print('loss:{}'.format(loss.item()))
        clip_grad_value_(model.parameters(),clip)
        opt.step()
        opt.zero_grad()
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)

iter:0, score:3.6730186227804276
set size:391
loss:0.6178246736526489
loss:0.5879775285720825
loss:0.687744140625
iter:1, score:3.589432654828933
set size:701
loss:0.4369705617427826
loss:0.20058569312095642
loss:0.17942233383655548
iter:2, score:3.5296665223040296
set size:1010
loss:0.2438039928674698
loss:0.12203182280063629
loss:0.11470020562410355
iter:3, score:3.548722390645303
set size:1295
loss:0.12401027232408524
loss:0.23123383522033691
loss:0.13625021278858185
iter:4, score:3.530532698137724
set size:1488
loss:0.13243801891803741
loss:0.1104731485247612
loss:0.1925094574689865
iter:5, score:3.517973148549156
set size:1626
loss:0.07968637347221375
loss:0.07447819411754608
loss:0.07772446423768997
iter:6, score:3.5101775660459085
set size:1768
loss:0.12764526903629303
loss:0.1285477578639984
loss:0.1284816712141037
iter:7, score:3.5084452143785203
set size:1914
loss:0.058054618537425995
loss:0.1528402864933014
loss:0.11438105255365372
iter:8, score:3.5071459506279785
set size:2

KeyboardInterrupt: 

In [5]:
torch.save(model.state_dict(), 'RL0.pt')

In [6]:
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6,
                         eps=0)

3.437418796015592

In [4]:
# model trained from start
# eps greedy
lr = 6e-5
clip = 0.03
epochs = 3
damp = 1
iterations = 30
max_data = 2000
states = set()
opt = Adam(model.parameters(),betas=(0.85, 0.95),lr=lr)
for i in range(iterations):
    # policy evaluation
    model.eval()
    score = evaluate_save(matrix,np.arange(2309),policy_model_eps,2309,0,model=model,words_embed=words_embed,top=0.6,
                         eps=0.25/(i+1)**(0.45))
    print('iter:{}, score:{}'.format(i,score))
    
    # prepare data for training
    index,val,log_prob = zip(*out)
    for element in index:
        states.add(tuple(element))
    print('set size:{}'.format(len(states)))
    prob = np.array(log_prob)
    prob -= prob.mean()
    prob = np.exp(prob/damp)
    prob = torch.tensor(prob,dtype=torch.float32,device='cuda')
    length = torch.tensor([len(i) for i in index],dtype=torch.float32,device='cuda')
    words = torch.tensor(words_embed[np.concatenate(index)]).long().to('cuda')
    ys = torch.tensor(val,dtype=torch.float32,device='cuda')

    
    # policy update
    model.train()
    for _ in range(epochs):
        yhat = model((words,length))
        loss = torch.sum(prob*(ys-yhat)**2)/torch.sum(prob)
        loss.backward()
        print('loss:{}'.format(loss.item()))
        clip_grad_value_(model.parameters(),clip)
        opt.step()
        opt.zero_grad()
    #print(model.w)
    while len(out)>max_data:
        out.pop(0)

iter:0, score:3.538328280640971
set size:376
loss:0.7447549700737
loss:0.6444686651229858
loss:0.5548148155212402
iter:1, score:3.503248159376356
set size:573
loss:0.4284925162792206
loss:0.35316458344459534
loss:0.24638698995113373
iter:2, score:3.4750974447812917
set size:646
loss:0.16492819786071777
loss:0.23509176075458527
loss:0.20920981466770172
iter:3, score:3.4854915547856242
set size:741
loss:0.1766650527715683
loss:0.19609062373638153
loss:0.0998220443725586
iter:4, score:3.473365093113904
set size:800
loss:0.10011761635541916
loss:0.17471493780612946
loss:0.09962547570466995
iter:5, score:3.4460805543525352
set size:813
loss:0.09312407672405243
loss:0.08883702754974365
loss:0.12613023817539215
iter:6, score:3.4447812906019952
set size:828
loss:0.1209559366106987
loss:0.05835673213005066
loss:0.048983290791511536
iter:7, score:3.4595062797747964
set size:883
loss:0.06940395385026932
loss:0.10452058911323547
loss:0.06105029210448265
iter:8, score:3.435253356431357
set size:886

In [5]:
torch.save(model.state_dict(), 'RL_eps.pt')

In [ ]:
evaluate(matrix,np.arange(2309),policy_model_eps,2309,model=model,words_embed=words_embed,top=0.6,
                         eps=0)

In [2]:
max_ = -1
for k,v in mapping.items():
    if v>max_:
        k_max = k
        max_ = v

In [4]:
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_lookup,len(k_max))

2.1029411764705874

In [4]:
max_

2.1029411764705874

In [8]:
# slightly worse than # of steps required by mapping as expected
evaluate(matrix[:,np.array(k_max)],np.array(k_max),policy_model,len(k_max),model=model,words_embed=words_embed,top=0.6)

2.1249999999999987

In [2]:
# entropy_best policy
    steps = evaluate(matrix,np.arange(2309),policy_entropy_best,2309,factor=i)
    print('factor:{}, steps:{}'.format(i,steps))

factor:0.0, steps:3.465569510610656
factor:0.5, steps:3.4638371589432673
factor:1.0, steps:3.4634040710264204
factor:1.5, steps:3.4634040710264204
factor:2.0, steps:3.4634040710264204
factor:2.5, steps:3.462970983109573
factor:3.0, steps:3.4638371589432673
factor:3.5, steps:3.4642702468601136


KeyboardInterrupt: 

In [2]:
# entropy policy
for i in np.linspace(0,7,15):
    steps = evaluate(matrix,np.arange(2309),policy_entropy,2309,factor=i)
    print('factor:{}, steps:{}'.format(i,steps))

factor:0.0, steps:3.5275010827197937
factor:0.5, steps:3.4638371589432673
factor:1.0, steps:3.4634040710264204
factor:1.5, steps:3.4634040710264204
factor:2.0, steps:3.4634040710264204
factor:2.5, steps:3.462970983109573
factor:3.0, steps:3.4638371589432673
factor:3.5, steps:3.4642702468601136


KeyboardInterrupt: 

In [2]:
# entropy policy with depth info
for i in np.linspace(1,2,5):
    steps = evaluate_depth(matrix,np.arange(2309),policy_entropy_depth,2309,factor=i,depth=1)
    print('factor:{}, steps:{}'.format(i,steps))

factor:1.0, steps:3.4634040710264204
factor:1.25, steps:3.462970983109573
factor:1.5, steps:3.4638371589432673
factor:1.75, steps:3.4642702468601136
factor:2.0, steps:3.4655695106106554


In [5]:
# see how well model does on full problem that it is not trained on
#     if count == 1: return 1
#     if count == 2: return 1.5
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.5266349068860987

In [6]:
# if entro == best:return row
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.8778692074491117

In [5]:
#                 if c == 1:
#                     continue
#                 if c == 2:
#                     value += p
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.6223473365093115

In [5]:
# old policy_model
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.558683412732789

In [5]:
# new policy_model
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.7756604590731935

In [7]:
# new + if entro==best: return row
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.873538328280638

In [10]:
# new policy_model +
#                 if c == 1:
#                     continue
#                 if c == 2:
#                     value += p
evaluate(matrix,np.arange(2309),policy_model,2309,model=model,words_embed=words_embed,top=0.6)

3.5240363793850142